In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import os
import sys
import matplotlib.pyplot as plt

In [2]:
deeplabv3plus_srcdir="./src"
sys.path.append(deeplabv3plus_srcdir)

In [4]:
#from image_utils import make_pascal_voc_label_csv
#make_pascal_voc_label_csv()

In [5]:
gpu_options = tf.compat.v1.GPUOptions(visible_device_list="1", allow_growth=True)
config = tf.compat.v1.ConfigProto(gpu_options = gpu_options)
tf.compat.v1.enable_eager_execution(config=config)

In [6]:
from model import deeplab_v3plus_transfer_os16
#from image_utils import make_x_from_image_paths,make_y_from_image_paths,convert_y_to_image_array
from data_gen import DataGenerator
from metrics import IoU
from label import Label

In [7]:
#import importlib
#importlib.reload(sys.modules['image_utils'])

In [8]:
out_dir = "TEST"
os.makedirs(out_dir, exist_ok=True)

In [9]:
seg_img_dir = "../pascal_voc_2012_datasets/VOCdevkit/VOC2012/SegmentationClass"
img_dir = "../pascal_voc_2012_datasets/VOCdevkit/VOC2012/JPEGImages"
train_set_path = "../pascal_voc_2012_datasets/VOCdevkit/VOC2012/ImageSets/Segmentation/train.txt"
valid_set_path = "../pascal_voc_2012_datasets/VOCdevkit/VOC2012/ImageSets/Segmentation/val.txt"

with open(train_set_path) as f:
    train_img_names = f.read().split("\n")[:-1]
with open(valid_set_path) as f:
    valid_img_names = f.read().split("\n")[:-1]

train_x_paths = np.array([os.path.join(img_dir,train_img_names[i]) + ".jpg" for i in range(len(train_img_names))])
train_y_paths = np.array([os.path.join(seg_img_dir,train_img_names[i]) + ".png" for i in range(len(train_img_names))])

valid_x_paths = np.array([os.path.join(img_dir,valid_img_names[i]) + ".jpg" for i in range(len(valid_img_names))])
valid_y_paths = np.array([os.path.join(seg_img_dir,valid_img_names[i]) + ".png" for i in range(len(valid_img_names))])

In [10]:
label_file_path = "./pascal_voc_label.csv"
label = Label(label_file_path)
image_size = (512,512)

In [11]:
batch_size=16
n_epochs=300

In [12]:
encoder = keras.applications.Xception(input_shape=(512,512,3), weights="imagenet", include_top=False)
preprocess = keras.applications.xception.preprocess_input
layer_name_to_decoder = "block3_sepconv2_bn"
encoder_end_layer_name = "block13_sepconv2_bn"
model = deeplab_v3plus_transfer_os16(label.n_labels, encoder, layer_name_to_decoder, encoder_end_layer_name)

W1205 14:20:39.908504 140736200002768 module_wrapper.py:139] From ./src/model.py:222: The name tf.image.resize_nearest_neighbor is deprecated. Please use tf.compat.v1.image.resize_nearest_neighbor instead.



in decoder, layer from encoder is resized from (?, 127, 127, 256) to (?, 128, 128, 256)


In [13]:
train_data_gen = DataGenerator(train_x_paths, train_y_paths, image_size, label, batch_size, preprocess, augmentation=True, shuffle=True, data_type="index_png")
valid_data_gen = DataGenerator(valid_x_paths, valid_y_paths, image_size, label, batch_size, preprocess, augmentation=False, shuffle=False, data_type="index_png")

In [14]:
#model.summary(line_length=150)

In [15]:
loss_function = tf.keras.losses.categorical_crossentropy
opt = tf.keras.optimizers.Adam()
model.compile(optimizer=opt, loss=loss_function, metrics=[IoU])

In [16]:
filepath = os.path.join(out_dir,'{epoch:06d}.h5')
cp_cb = keras.callbacks.ModelCheckpoint(filepath, 
                                        monitor='val_IoU', 
                                        verbose=1, 
                                        save_best_only=True, 
                                        save_weights_only=False, 
                                        mode='max')

In [17]:
#hist = model.fit_generator(data_gen, validation_data=(valid_x, valid_y), epochs=par.n_epochs, steps_per_epoch=par.n_batch, callbacks=[cp_cb])
hist = model.fit_generator(train_data_gen,
                           epochs=n_epochs,
                           steps_per_epoch=len(train_data_gen),
                           validation_data=valid_data_gen,
                           validation_steps=len(valid_data_gen),
                           #shuffle = False,
                           workers=8,
                           use_multiprocessing=True,
                           callbacks=[cp_cb])
#hist = model.fit_generator(data_gen, epochs=par.n_epochs, steps_per_epoch=par.n_batch, workers=8, use_multiprocessing=True)

Epoch 1/300


W1205 14:20:57.548191 140736200002768 deprecation.py:323] From /home/taniguchi-j/anaconda3/envs/ppc/lib/python3.6/site-packages/tensorflow_core/python/ops/math_grad.py:1394: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


18/92 [====>.........................] - ETA: 2:44 - loss: 2.6508 - IoU: 0.0046

Process Keras_worker_ForkPoolWorker-3:
Process Keras_worker_ForkPoolWorker-1:
Process Keras_worker_ForkPoolWorker-6:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/taniguchi-j/anaconda3/envs/ppc/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/taniguchi-j/anaconda3/envs/ppc/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/taniguchi-j/anaconda3/envs/ppc/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/taniguchi-j/anaconda3/envs/ppc/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/home/taniguchi-j/anaconda3/envs/ppc/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/taniguchi-j/anaconda3/envs/ppc/lib/python3.6/site-packages/tensorflow_core/python/keras/utils/data_

KeyboardInterrupt: 

Process Keras_worker_ForkPoolWorker-2:
Traceback (most recent call last):
  File "/home/taniguchi-j/anaconda3/envs/ppc/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/taniguchi-j/anaconda3/envs/ppc/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/taniguchi-j/anaconda3/envs/ppc/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/home/taniguchi-j/anaconda3/envs/ppc/lib/python3.6/site-packages/tensorflow_core/python/keras/utils/data_utils.py", line 571, in get_index
    return _SHARED_SEQUENCES[uid][i]
  File "./src/data_gen.py", line 47, in __getitem__
    x,y = data_augment(x,y,image_size=self.image_size, p=0.95)
  File "./src/data_augment.py", line 42, in data_augment
    auged = aug(image=images[i,:,:,:],mask=masks[i,:,:,:])
  File "/home/taniguchi-j/anaconda3/envs/ppc/lib/python3.6/site-packages/albumentations/

In [ ]:
plt.figure(figsize=(30,10))

plt.subplot(1,3,1)
plt.plot(hist.history["loss"], label="loss")
plt.plot(hist.history["val_loss"], label="val_loss")
plt.legend()

plt.subplot(1,3,2)
plt.plot(hist.history["IoU"], label="IoU")
plt.plot(hist.history["val_IoU"], label="val_IoU")
plt.legend()
plt.savefig(os.path.join(out_dir,'losscurve.png'))

In [ ]:
model.save(os.path.join(out_dir,'final_epoch.h5'))
for key in sorted(hist.history.keys()):
    np.savetxt(os.path.join(out_dir,key+'.txt'),np.array(hist.history[key]))
